# LangChain: Evaluation

When building an LLM-based application, evaluating the model's performance using clear and measurable metrics is a critical and often challenging step. These metrics not only help assess how well the model performs but also allow us to quantify the impact of any changes made to the application—such as integrating a Vector database—by determining whether these changes improve or degrade the application's overall performance.

This notebook explores some frameworks to evaluate LLM based applications.

## Notebook Outline:

* Example generation
* Manual evaluation (and debuging)
* LLM-assisted evaluation
* LangChain evaluation platform

## 1. Load and import required libraries

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [3]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [4]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.embeddings import OpenAIEmbeddings

## 2. Create our QandA application

In [5]:
file = r"..\Data\OutdoorClothingCatalog_1000.csv"
loader = CSVLoader(file_path=file)
data = loader.load()

What the Code Does:
- Document Embedding: The code takes documents from the `loader`, processes them into text, and converts them into vector embeddings using the specified `embeddings` model.
- Index Creation: The embeddings are stored in a `DocArrayInMemorySearch` vector store, creating an index that can be queried for similarity-based search.
- Search-Ready: The resulting `index` can now be used to perform tasks like question-answering, semantic search, or retrieving relevant documents based on input queries.

Breakdown:
1. **`VectorstoreIndexCreator`**:  
   - This is a utility in LangChain that simplifies the process of creating an index for vector-based search. It manages the pipeline for embedding documents, storing them in a vector store, and making them searchable.

2. **`embedding=embeddings`**:  
   - This specifies the embedding model to be used.  
   - The `embeddings` variable likely holds a pre-trained embedding model (e.g., OpenAI's embeddings, Sentence Transformers, etc.), which converts text into numerical vectors for similarity search.

3. **`vectorstore_cls=DocArrayInMemorySearch`**:  
   - `DocArrayInMemorySearch` is a vector store implementation. It stores document embeddings in memory, making it fast for small to medium datasets.  
   - This is a lightweight, in-memory solution for vector storage and similarity search, part of DocArray's library.

4. **`.from_loaders([loader])`**:  
   - The `.from_loaders()` method is used to populate the index with documents.  
   - `loader` is likely an instance of a document loader (e.g., `TextLoader`, `DirectoryLoader`, etc.), which reads documents from a file, directory, or other sources and prepares them for indexing.  
   - The `loader` provides the text data that gets embedded and stored in the vector store.

In [6]:
# Instantiating embeddings model 
embeddings = OpenAIEmbeddings()

index = VectorstoreIndexCreator(
    embedding=embeddings,
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

C:\Users\rg255041\AppData\Local\Temp\ipykernel_21168\575871369.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()
c:\Users\rg255041\AppData\Local\anaconda3\envs\Langchain\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


Create the pipeline

In [7]:
llm = ChatOpenAI(temperature = 0.0, model=llm_model)
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=index.vectorstore.as_retriever(), 
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

C:\Users\rg255041\AppData\Local\Temp\ipykernel_21168\1284373331.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature = 0.0, model=llm_model)


## 3. Test the pipeline

In [8]:
data[10]

Document(metadata={'source': '..\\Data\\OutdoorClothingCatalog_1000.csv', 'row': 10}, page_content=": 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\n\nSize & Fit\n- Pants are Favorite Fit: Sits lower on the waist.\n- Relaxed Fit: Our most generous fit sits farthest from the body.\n\nFabric & Care\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features\n- Relaxed fit top with raglan sleeves and rounded hem.\n- Pull-on pants have a wide elastic waistband and drawstring, side pockets and a modern slim leg.\n\nImported.")

In [9]:
data[11]

Document(metadata={'source': '..\\Data\\OutdoorClothingCatalog_1000.csv', 'row': 11}, page_content=': 11\nname: Ultra-Lofty 850 Stretch Down Hooded Jacket\ndescription: This technical stretch down jacket from our DownTek collection is sure to keep you warm and comfortable with its full-stretch construction providing exceptional range of motion. With a slightly fitted style that falls at the hip and best with a midweight layer, this jacket is suitable for light activity up to 20° and moderate activity up to -30°. The soft and durable 100% polyester shell offers complete windproof protection and is insulated with warm, lofty goose down. Other features include welded baffles for a no-stitch construction and excellent stretch, an adjustable hood, an interior media port and mesh stash pocket and a hem drawcord. Machine wash and dry. Imported.')

Hard-coded examples to build source truth

In [42]:
examples = [
    {
        "query": "Do the Cozy Comfort Pullover Set\
        have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty \
        850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

LLM-Generated examples (automated) to build source truth

In [11]:
from langchain.evaluation.qa import QAGenerateChain

`QAGenerateChain` will take documents as input and **create question and answer pairs automatically** from each document. It does this with a specified LLM, OpenAI LLM in this case.

In [12]:
example_gen_chain = QAGenerateChain.from_llm(ChatOpenAI(model=llm_model))

Apply the method and parse the output into a dict, instead of a single sting. 

In [30]:
# the warning below can be safely ignored
new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[:5]]
)

c:\Users\rg255041\AppData\Local\anaconda3\envs\Langchain\Lib\site-packages\langchain\chains\llm.py:369: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Check query and answer pair together with actual document. 

In [14]:
new_examples[0]

{'qa_pairs': {'query': "What is the approximate weight of the Women's Campside Oxfords per pair?",
  'answer': "The approximate weight of the Women's Campside Oxfords per pair is 1 lb. 1 oz."}}

In [15]:
data[0]

Document(metadata={'source': '..\\Data\\OutdoorClothingCatalog_1000.csv', 'row': 0}, page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.")

Here, we combine the LLM generated example with the manually created example. 

In [43]:
examples += new_examples

Pass one of the prompts to get a response

In [17]:
qa.run(examples[0]["query"])

C:\Users\rg255041\AppData\Local\Temp\ipykernel_21168\1223946598.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qa.run(examples[0]["query"])




> Entering new RetrievalQA chain...

> Finished chain.


'Yes, the Cozy Comfort Pullover Set does have side pockets.'

## 4. Manual Evaluation

In the above section, we have generated a response but it is not clear what the prompt was, making it very difficult to evaluate the performance. If this was a more complex chain,  we are also not able to see the intermediate results. 

Therefore, the section below shows how we can get visibility into the prompt and documents retrieved with `langchain.debug = True`.

In [18]:
import langchain
langchain.debug = True

Re-running the above query shows a lot more results. We can see that it first goes to a `RetrievalQA` chain and then a `StuffDocumentsChain` chain, finally by the `LLMChain` which has the original prompt  and the context that was created from a number of documents that was retrieved. We can also see in the `system` message the underlying prompt that the `RetrievalQA` chain uses to generate the Q&A. "Use the following pieces of context to answer the user's question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer."

We also see important metadata about the query such as the token usage at the end. This will be important to track with LLMs over time as it will correspond very closely to cost. 

In [19]:
qa.run(examples[0]["query"])

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Do the Cozy Comfort Pullover Set        have side pockets?"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "Do the Cozy Comfort Pullover Set        have side pockets?",
  "context": ": 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\n\nSize & Fit\n- Pants are Favorite Fit: Sits lower on the waist.\n- Relaxed Fit: Our most generous fit sits farthest from the body.\n\nFabric & Care\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features\n- Relaxed fit top with raglan sleeves and rounded hem.\n- Pull-

'Yes, the Cozy Comfort Pullover Set does have side pockets.'

In [20]:
# Turn off the debug mode
langchain.debug = False

In [50]:
# Clear List
examples = []

# Add Manual Q&A pair
examples = [
    {
        "query": "Do the Cozy Comfort Pullover Set\
        have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty \
        850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

# Apply the example generation chain
raw_examples = example_gen_chain.apply_and_parse([{"doc": t} for t in data[:5]])

# Extract and flatten the nested 'qa_pairs' structure
new_examples = [example['qa_pairs'] for example in raw_examples]

examples += new_examples

c:\Users\rg255041\AppData\Local\anaconda3\envs\Langchain\Lib\site-packages\langchain\chains\llm.py:369: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


In [51]:
print(examples[2])

{'query': "What is the approximate weight of one pair of Women's Campside Oxfords?", 'answer': "The approximate weight of one pair of Women's Campside Oxfords is 1 lb. 1 oz."}


## 5. LLM assisted evaluation

The above example shows how we can manually evaluate the output by looking at the `debug` logs generated when running the query. In this section, instead of running the chain on all outputs, we leverage an LLM for this task. 

In [52]:
predictions = qa.apply(examples)



> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


In [27]:
from langchain.evaluation.qa import QAEvalChain

In [28]:
llm = ChatOpenAI(temperature=0, model=llm_model)
eval_chain = QAEvalChain.from_llm(llm)

In [29]:
graded_outputs = eval_chain.evaluate(examples, predictions)

NameError: name 'predictions' is not defined

In [30]:
for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print("Predicted Grade: " + graded_outputs[i]['text'])
    print()

Example 0:


NameError: name 'predictions' is not defined

In [31]:
graded_outputs[0]

NameError: name 'graded_outputs' is not defined